In [ ]:

from astropy.io import fits, ascii 
from astropy.table import Column
from astropy import units as u
from scipy import optimize
from os.path import expanduser
# from ROOT import TRolke

#%matplotlib infloat(line
%pylab inline


In [ ]:
home = expanduser("~")
gc_dir = home + "/Dropbox/GalacticCenter/"


In [ ]:
erg2TeV = (u.erg).to(u.TeV)
print(erg2TeV)

pylab.rcParams['figure.figsize'] = (12.0, 6.0)


In [ ]:
# define our line fitting function
fitfunc = lambda p, x: p[0] + p[1] * (x)
errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err

# define our () fitting function
fitfuncECPL = lambda p, x: p[0] + p[1] * np.log(x) - (x) / p[2] # np.log is natural log  
errfuncECPL = lambda p, x, y, err: (np.log(y) - fitfuncECPL(p, x)) / (err)

fitfuncECPL_CF = lambda N0, gamma, beta, E: N0 + gamma*E - 1.*np.exp(E) / beta 


#these are just copied from http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_tutorial.html
f = lambda E, N0, E0, gamma: N0*(E/E0)**(-1.*gamma)
ferr = lambda E, F, N0, N0err, E0, cov_gg: F*np.sqrt(N0err**2/N0**2 \
                                                     + ((np.log(E/E0))**2)*cov_gg)

f_ecpl = lambda E,N0,E0,gamma,beta: N0*(E/E0)**(-1.*gamma)*np.exp(-1.*E/beta)
ferr_ecpl = lambda E, F, N0, N0err, E0, cov_gg, b, cov_bb: \
F*np.sqrt(N0err**2/N0**2 + ((np.log(E/E0))**2) * cov_gg + (E/E0)**2 / b**4 * cov_bb)

def plotPythonFit(energy, flux, flux_err_arg, color, cuts, ECPL=False, E0=1.):
    """fit flux points to a curve then plot"""
    logx = np.log(energy/E0)
    logy = np.log(flux)
    #logyerr = np.log(flux_err_arg)
    
    if isinstance(flux_err_arg, tuple):
        flux_err = (flux_err_arg[1] + flux_err_arg[0]) / 2 
    else:
        flux_err = flux_err_arg 

    if ECPL:
        pinit = [-26, -2.25, 10]
        out = optimize.leastsq(errfuncECPL, pinit,
                               args=(energy/E0, flux, flux_err / flux), 
                               full_output=1)

    else:
        pinit = [-26, -2.25] # nb ln
    
        out = optimize.leastsq(errfunc, pinit,
                               args=(logx, logy, logyerr), 
                               full_output=1)
    # end if else ECPL 

    pfinal = out[0]
    covar  = out[1]
    print("parameters:")
    print(pfinal)
    print("covariance matrix")
    print(covar)
    
    N0    = np.exp(pfinal[0])
    gamma = pfinal[1]
    E     = np.linspace(energy[0], energy[-1], num=100)
    
    if ECPL:
        beta = pfinal[2]
        
        F    = f_ecpl(E, N0, E0, -1.*gamma, beta)
        chi2 = np.sum((flux - f_ecpl(energy, N0, E0, -1.*gamma, beta))**2/flux_err**2) / (len(energy) - 3)
        print("chi^2: "+str(chi2)+'\n')
        beta_err  = np.sqrt( covar[2][2] ) * chi2 #* N0  
        gamma_err = np.sqrt( covar[0][0] ) * chi2
        N0_err    = np.sqrt( covar[1][1] ) * N0 * chi2
        cov_gg = gamma_err**2
        cov_bb = beta_err**2
        Ferr = ferr_ecpl(E, F, N0, N0_err, E0, cov_gg, beta, cov_bb)

        fitTitle = (cuts + ' - N0: {0:.2e} +- {2:.2e}, '\
                    'G: {1:.2f} +- {3:.2f}, '\
                    'Ec: {4:.2f} +- {5:.2f}, '
                    'E0: {6:.0f}').format(float(N0), float(gamma),
                                          float(N0_err), float(gamma_err),
                                          float(beta), float(beta_err), float(E0))

    else:
        F    = f(E, N0, E0, -1.*gamma)
        chi2 = np.sum((flux - f(energy, N0, E0, -1.*gamma))**2 / flux_err**2) / (len(energy) - 2)
        print(chi2)
        gamma_err = np.sqrt( covar[0][0] ) * chi2
        N0_err    = np.sqrt( covar[1][1] ) * N0 * chi2
        cov_gg = gamma_err**2
        Ferr = ferr(E, F, N0, N0_err, E0, cov_gg)
        
        fitTitle = (cuts + ' - N0: {0:.2e} +- {2:.2e}, '\
                    'gamma: {1:.2f} +- {3:.2f}, '\
                    'E0: {4:.2f}').format(float(N0), float(gamma),
                                          float(N0_err), float(gamma_err), float(E0))

    plt.plot(E, F * (E)**power, c=color, ls="--", marker="", label = fitTitle)
    plt.fill_between(E, (E)**power*(F+Ferr), (E)**power*(F-Ferr), color=color, alpha='0.25')

# end plotPythonFit 


# HESS Points

In [ ]:
# SgrA spectral points and errors 
with open(gc_dir+"spectralPoints/HESS_SgrAstar_SpectralPoints_Bins.txt") as infile:
    H_energy = np.array([])
    H_flux = np.array([])
    H_err_up = np.array([])
    H_err_dwn = np.array([])
    for line in iter(infile):
        line = line.split()
        try:
            float(line[0]) and float(line[3]) and float(line[4]) and float(line[5])
            H_energy = np.append(H_energy, float(line[0]))
            H_flux = np.append(H_flux, float(line[3]))
            H_err_up = np.append(H_err_up, float(line[4]))
            H_err_dwn = np.append(H_err_dwn, float(line[5]))
        except ValueError:
            continue 
# end extract SgrA spectral points from file     

#"/spectralPoints/HESS_Diffuse_SpectralPoints_Bins.txt"

In [ ]:
# dN/dE
power = 0.
plotPythonFit(H_energy[:-3], H_flux[:-3], (H_err_dwn[:-3],H_err_up[:-3]), 
              "blue", "HESS", ECPL=True)
# flux points
plt.errorbar(H_energy[:-3], H_flux[:-3], yerr=(H_err_dwn[:-3],H_err_up[:-3]), 
             marker="+", ls="", color="blue", label="HESS")
# upper limit points
plt.errorbar(H_energy[-3:], H_flux[-3:], yerr=(H_err_up[-3:],H_err_dwn[-3:]), 
             marker="_", ls="", uplims=True, color = "blue")

plt.loglog(nonposy="clip")
plt.legend(loc="best")
plt.xlabel("Energy [TeV]")
plt.ylabel(r"dN/dE [TeV$^{-1}$ cm$^{-2}$ s$^{-1}$]")

In [ ]:
# E^2 dN/dE
power = 2.
plotPythonFit(H_energy[:-3], H_flux[:-3], (H_err_dwn[:-3],H_err_up[:-3]), 
              "blue", "HESS", ECPL=True)
# flux points
plt.errorbar(H_energy[:-3], H_energy[:-3]**2*H_flux[:-3], 
             yerr=H_energy[:-3]**2*(H_err_dwn[:-3], H_err_up[:-3]), 
             marker="+", ls="", color="blue", label="HESS")
# upper limit points
plt.errorbar(H_energy[-3:], H_energy[-3:]**2*H_flux[-3:], 
             yerr=(H_energy[-3:]**2*H_err_up[-3:], H_energy[-3:]**2*H_err_dwn[-3:]), 
             marker="_", ls="", uplims=True, color = "blue")

plt.loglog(nonposy="clip")
plt.legend(loc="best")
plt.xlabel("Energy [TeV]")
plt.ylabel(r"E$^2$ dN/dE [TeV cm$^{-2}$ s$^{-1}$]")


In [ ]:
VEGAS_Points = '''
2.813 3.44e-13 4.52e-14 
3.541 2.17e-13 2.43e-14
4.458 1.23e-13 1.37e-14
5.613 5.13e-14  7.4e-15
7.066 2.72e-14  4.3e-15
8.896 1.27e-14 2.48e-15
12.49  5.8e-15 8.86e-16
 19.8 1.44e-15 3.36e-16
31.39 1.22e-16 8.14e-17'''

V_Points = ascii.read(VEGAS_Points)
V_Points

In [ ]:
plotPythonFit(V_Points['col1'], V_Points['col2'], V_Points['col3'], "blue", "HESS", ECPL=True, E0=1.25)
plt.errorbar(V_Points['col1'], V_Points['col2'] * V_Points['col1']**2, 
             yerr = V_Points['col3'] * V_Points['col1']**2,
            label = "VERITAS 2016 Paper", ls = "", marker = "+", color="red")
plt.loglog(nonposy="clip")
plt.ylim(ymin=1e-13)